In [134]:
#class="dpf__sc-2arhs5-0 gRshUo"
import pandas as pd
import os
import re
import csv
import requests
from bs4 import BeautifulSoup
my_dir = os.path.dirname(os.path.abspath('__file__'))
os.chdir(my_dir)

In [135]:
headers = {
    'authority': 'www.zillow.com',
    'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36',
    'accept': '*/*',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.zillow.com/los-angeles-ca/2_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A2%7D%2C%22usersSearchTerm%22%3A%22Los%20Angeles%22%2C%22mapBounds%22%3A%7B%22west%22%3A-118.88620504589845%2C%22east%22%3A-117.93726095410157%2C%22south%22%3A33.54554919445917%2C%22north%22%3A34.49481997322805%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A12447%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%7D',
    'accept-language': 'en-US,en;q=0.9',
}

files = ['data/uncleaned data/local data/zillow_links_data_pnw.csv']

In [136]:
# This function will open a csv containing zillow listing links and scrape certain information

def zillow_link(myfile):
    with open(myfile, 'r') as csv_file:
        
        # Create a dataframe we will use to dump the info
        columns = ['city', 
            'state', 
            'zip', 
            'address', 
            'type', 
            'year built', 
            'heat type', 
            'cooling type', 
            'garage type',
            'HOA cost', 
            'sqft', 
            'price per sqft', 
            'buyers fee', 
            'listing price']
        df = pd.DataFrame(columns=columns)
        expected_prefix = "https://www.zillow.com/homedetails/"
        
        # Open the csv
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            link = row[0]
            # Error handling for valid links only
            if not link.startswith(expected_prefix):
                print(f"Invalid URL: {link}. Skipping...")
                continue

            # Get the website HTML and parse it
            response = requests.get(link, headers=headers)
            print(link)
            print(response.status_code)
            if response.status_code == 200:
                html_content = response.text
                soup = BeautifulSoup(html_content, 'html.parser')
                property_info = []
                
                # Get the basic property address bits from the link itself
                parts = link.rstrip('/').split('/')
                address_part = parts[-2]
                address_parts = address_part.split('-')
                zip_code = address_parts[-1]
                state = address_parts[-2]
                city = address_parts[-3]
                address = "-".join(address_parts[:-3])
                
                # The text elements here are the listing of what the house has. Sometimes it has HOA info, sometimes it doesn't
                # So I treat length of 9 as having HOA, and 8 as not
                text_elements = soup.find_all('span', class_='Text-c11n-8-84-3__sc-aiai24-0 dpf__sc-2arhs5-3 hrfydd kOlNqB')
                if len(text_elements) == 9:
                    print('saving...')
                    # Save the previous info to our list
                    property_info.append(city)
                    property_info.append(state)
                    property_info.append(zip_code)
                    property_info.append(address)
                    # Get the text we scraped and add it to our list
                    for element in text_elements:
                        property_info.append(element.text)
                    #Get the price info from elsewhere on the page
                    price_element = soup.find('span', {'data-testid': 'price'})
                    price_text = price_element.span.get_text()
                    price_num = int(price_text.replace('$', '').replace(',', ''))
                    property_info.append(price_num)
                    
                    # Print the extracted property information for debugging
                    print('text_el 9',len(property_info),property_info)
                    
                    # Add the info to our dataframe
                    if len(property_info) == len(columns):
                        df.loc[len(df)] = property_info
                    else:
                        print("Length mismatch! Cannot append to DataFrame.")
                
                # Same as above but without HOA info
                if len(text_elements) == 8:
                    print('saving...')
                    property_info.append(city)
                    property_info.append(state)
                    property_info.append(zip_code)
                    property_info.append(address)
                    
                    #Here I add info to our list but add HOA price of 0 in the correct spot
                    for index, element in enumerate(text_elements):
                        property_info.append(element.text)
                        if index == 4:
                            property_info.append('0')
                    price_element = soup.find('span', {'data-testid': 'price'})
                    price_text = price_element.span.get_text()
                    price_num = int(price_text.replace('$', '').replace(',', ''))
                    property_info.append(price_num)
                    # Print the extracted property information
                    print('text_el 8', len(property_info),property_info)
                    
                    # Add it all to our dataframe
                    if len(property_info) == len(columns):
                        df.loc[len(df)] = property_info
                    else:
                        print("Length mismatch! Cannot append to DataFrame.")
            else:
                print(f"Failed to retrieve HTML content for {link}")
    
    # Return this dataframe, end of function
    return df


In [137]:
df_list = []
for file in files:
    temp_df = zillow_link(file)
    df_list.append(temp_df)
df = pd.concat(df_list, ignore_index=True)
    

Invalid URL: link. Skipping...
https://www.zillow.com/homedetails/2655-NW-Highland-Dr-UNIT-83-Corvallis-OR-97330/52641226_zpid/
200
https://www.zillow.com/homedetails/6425-NW-Sisters-Pl-Corvallis-OR-97330/48207026_zpid/
200
https://www.zillow.com/homedetails/29418-Highway-34-Corvallis-OR-97333/209756242_zpid/
200
saving...
text_el 8 14 ['Corvallis', 'OR', '97333', '29418-Highway-34', 'Single family residence', 'Built in 1935', 'Natural gas', 'No data', '2 Garage spaces', '0', '30.50 Acres', '$553 price/sqft', '2.5%\xa0buyers agency fee', 1350000]
https://www.zillow.com/homedetails/NW-Niagara-Dr-LOT-400-Corvallis-OR-97330/2063525467_zpid/
200
https://www.zillow.com/homedetails/5055-NE-Elliott-Cir-Corvallis-OR-97330/52642266_zpid/
200
https://www.zillow.com/homedetails/4865-SW-Philomath-Blvd-Corvallis-OR-97333/48198869_zpid/
200
saving...
text_el 8 14 ['Corvallis', 'OR', '97333', '4865-SW-Philomath-Blvd', 'Single family residence', 'Built in 1947', 'Natural gas, forced air', 'No data', '

In [138]:
#df_save = df

In [141]:
df

,city,state,zip,address,type,year built,heat type,cooling type,garage type,HOA cost,sqft,price per sqft,buyers fee,listing price
0,Corvallis,OR,97333,29418-Highway-34,Single family residence,Built in 1935,Natural gas,No data,2 Garage spaces,0,30.50 Acres,$553 price/sqft,2.5% buyers agency fee,1350000
1,Corvallis,OR,97333,4865-SW-Philomath-Blvd,Single family residence,Built in 1947,"Natural gas, forced air",No data,2 Attached garage spaces,0,1.63 Acres,$356 price/sqft,2.5% buyers agency fee,470000
2,Corvallis,OR,97330,2763-NW-Daylily-Ave,Condominium,Built in 2002,No data,No data,2 Attached garage spaces,0,$190 monthly HOA fee,$274 price/sqft,2.5% buyers agency fee,429900
3,Corvallis,OR,97333,4145-SW-Country-Club-Dr,Single family residence,Built in 2023,Forced air,Central air,2 Attached garage spaces,$100 annually HOA fee,0.26 Acres,$314 price/sqft,$6250 buyers agency fee,689624
4,Portland,OR,97218,4139-NE-62nd-Ave,"Residential, single family residence",Built in 1928,Mini split,Heat pump,Open parking,0,"4,791 sqft",$280 price/sqft,2.5% buyers agency fee,295000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,Bothell,WA,98021,4328-228th-St-SE-MC8,Single family residence,Built in 2023,"Fireplace(s), heat pump","90%+ high efficiency, heat pump",2 Attached garage spaces,$237 monthly HOA fee,"3,598 sqft",$589 price/sqft,2% buyers agency fee,1374995
984,Bothell,WA,98012,2804-193rd-Pl-SE-F12,Condominium,Built in 2023,"Fireplace(s), heat pump","90%+ high efficiency, heat pump",2 Garage spaces,0,Has HOA fee,$548 price/sqft,2% buyers agency fee,1279995
985,Snohomish,WA,98296,,Single family residence,Built in 1995,Fireplace(s),Central air,4 Garage spaces,0,2.44 Acres,$486 price/sqft,2.5% buyers agency fee,1500000
986,Bothell,WA,98012,2814-194th-Pl-SE-F19,Condominium,Built in 2023,"Fireplace(s), heat pump","90%+ high efficiency, heat pump",2 Garage spaces,0,Has HOA fee,$487 price/sqft,2% buyers agency fee,1349995


In [140]:
df.to_csv('data/uncleaned data/local data/zillow_listings_pnw.csv', index = False)